The train data has no labels. In this notebook i am extracting train labels.

In [1]:
import pandas as pd

In [2]:
df_train_tte = pd.read_csv('train_tte.csv')

In [3]:
df_train_readouts = pd.read_csv('train_operational_readouts.csv')

In [12]:
def process_datasets(train_tte, train_operational_readouts):
    train_tte['class_labels'] = None

    for index, row in train_tte.iterrows():
        vehicle_id = row['vehicle_id']
        length_of_study_time_step = row['length_of_study_time_step']
        in_study_repair = row['in_study_repair']

        vehicle_rows = train_operational_readouts[train_operational_readouts['vehicle_id'] == vehicle_id]

        if in_study_repair == 0:
            threshold_timestep = length_of_study_time_step - 48

            vehicle_rows_to_keep = vehicle_rows[vehicle_rows['time_step'] <= threshold_timestep]

            train_operational_readouts = train_operational_readouts[~((train_operational_readouts['vehicle_id'] == vehicle_id) & (train_operational_readouts['time_step'] > threshold_timestep))]

            train_tte.loc[index, 'class_labels'] = 0  
        else:  # in_study_repair == 1
            max_timestep = vehicle_rows['time_step'].max()
            diff = length_of_study_time_step - max_timestep

            if diff > 48:
                train_tte.loc[index, 'class_labels'] = 0
            elif 24 < diff <= 48:
                train_tte.loc[index, 'class_labels'] = 1
            elif 12 < diff <= 24:
                train_tte.loc[index, 'class_labels'] = 2
            elif 6 < diff <= 12:
                train_tte.loc[index, 'class_labels'] = 3
            elif 0 < diff <= 6:
                train_tte.loc[index, 'class_labels'] = 4

    return train_tte, train_operational_readouts


In [13]:
processed_tte, processed_operational_readouts = process_datasets(df_train_tte, df_train_readouts)

In [14]:
processed_tte.to_csv('processed_train_labels.csv', index=False)
print("Başarılı")

Başarılı


In [15]:
processed_operational_readouts4.to_csv('processed_train_operational_readouts.csv', index=False)
print("Başarılı")

Başarılı
